In [1]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
# di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

display(HTML("<style>.container { width:100% !important; }</style>"))

import hail as hl
hl.init()

Running on Apache Spark version 2.4.1
SparkUI available at http://633fc3f91d0f:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.29-cf730c8fc8f6
LOGGING: writing to /hail/large_vcf_filtering_annotation/hail-20191221-0627-0.2.29-cf730c8fc8f6.log


In [2]:
mt = hl.read_matrix_table('step4_output.mt')

In [9]:
mt.sample_rows(0.00002).gnomad_v3.transcript_consequences.gene_symbol.show()

locus,alleles,
locus<GRCh38>,array<str>,array<str>
chr1:16059952,"[""G"",""C""]","[""FAM131C"",""CLCNKB"",""CLCNKB"",""CLCNKB"",""FAM131C"",""CLCNKB""]"
chr1:20998212,"[""TACACACACACACAC"",""T""]","[""EIF4G3"",""EIF4G3"",""EIF4G3"",""EIF4G3"",""EIF4G3"",""EIF4G3"",""EIF4G3"",""EIF4G3""]"
chr1:109899065,"[""A"",""C""]","[""AL450468.2""]"
chr1:170227201,"[""A"",""G""]","[""LINC01681""]"
chr1:179691844,"[""C"",""CA""]","[""TDRD5"",""TDRD5"",""TDRD5"",""TDRD5""]"
chr1:223895728,"[""G"",""A""]",NA
chr1:228223554,"[""G"",""C""]","[""OBSCN"",""OBSCN"",""OBSCN"",""OBSCN"",""OBSCN""]"
chr1:234691072,"[""T"",""C""]","[""AL160408.3""]"
chr2:5713566,"[""C"",""T""]",NA


In [10]:
gwas = hl.read_table('./gwas/TS_Oct2018.mt')

In [11]:
gwas = gwas.transmute(variant = hl.parse_variant(hl.str(gwas.CHR)+':'+hl.str(gwas.BP)+':'+hl.str(gwas.A1)+':'+hl.str(gwas.A2), reference_genome='GRCh37'))

In [13]:
rg37 = hl.get_reference('GRCh37')  
rg38 = hl.get_reference('GRCh38')  
rg37.add_liftover('references_grch37_to_grch38.over.chain.gz', rg38)

In [16]:
gwas = gwas.annotate(new_locus=hl.liftover(gwas.variant.locus, 'GRCh38'))  
gwas = gwas.filter(hl.is_defined(gwas.new_locus))  
gwas = gwas.key_by(locus=gwas.new_locus, alleles=gwas.variant.alleles)

In [17]:
gwas=gwas.select(gwas.SNP, gwas.P)

In [18]:
mt = mt.annotate_rows(gwas_p = gwas[mt.row_key])

In [21]:
mt.write('step5_output.mt')

2019-12-21 06:57:23 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-21 06:57:35 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-21 07:23:00 Hail: INFO: wrote matrix table with 7615409 rows and 151 columns in 591 partitions to step5_output.mt
